# Lecture 11 – Taking and Filtering Rows

## Data 6, Summer 2022

In [ ]:
from datascience import *
import numpy as np

## SAT Data

Today we will be working with a dataset showing aggregated (average) SAT scores by state. This data is from 2014, so the total score is out of 2400 (over three sections each out of 800) instead of 1600.

In [ ]:
sat = Table.read_table('data/sat2014-lecture.csv')
sat

Recall the table methods and properties we can use to learn more above our data and even create new data:

In [ ]:
sat.num_rows, sat.num_columns

It would be nice to have a combined score too.

In [ ]:
sat.column('Critical Reading') + sat.column('Math') + sat.column('Writing')

In [ ]:
sat = sat.with_columns(
    'Combined', sat.column('Critical Reading') + sat.column('Math') + sat.column('Writing')
)

In [ ]:
sat

## `.take`

Sometimes, it is a little tricky to work with a large dataset (even though this specific dataset isn't _that_ large). To make it easier for us to understand parts of our data, we may want to just look at certain rows in our table. We can do that using `tbl.take(indices`, which takes in a single index or list of indices corresponding to the rows of the table we want to take.

In [ ]:
sat.take(2) # Use `tbl.take()` to take the third row in the `sat` table (Remember indices are 0-indexed)

In [ ]:
sat.take(make_array(1, 4, 3)) # Pass in an array with values 1, 4, and 3 to take the 2nd, 5th and 4th rows of `sat`

Recall that `np.arange()` makes it really easy to generate an array of sequential numbers. The [Data 6 Python Reference](https://data6.org/su22/reference/#numpy-array-functions) provides a good explanation of how `np.arange` works.

In [ ]:
np.arange(5)

We can pass array ranges into `.take()` just as we would manually-created arrays. This is often much easier that manually typing out arrays.

In [ ]:
sat.take(np.arange(5))

When we combine `sort` and `take`, we can get some pretty powerful answers.

What are the five states with the highest math scores?

In [ ]:
sat.sort('Math', descending = True).take(np.arange(5))

What are the top 8 states in terms of participation?

In [ ]:
sat.sort('Participation Rate', descending = True).take(np.arange(8))

Note: `.take` works on arrays too, not just tables!

In [ ]:
sat.column('State').take(np.arange(5))

In [ ]:
sat.take(np.arange(5)).column('State')

### Quick Check 1

In [ ]:
animals = Table.read_table('data/animals.csv')
animals

Using the `animals` table, fill in the blanks in the code below so that the result is an array containing the names of the five smallest animals by body weight, in increasing order.

In [ ]:
animals._____(_____).column(_____).take(_____) # Replace the blanks with your answer

## Booleans

Another Python data type is the `bool` or Boolean, whic only has two possible values: `True` and `False`.

In [ ]:
True

In [ ]:
f = False
f

In [ ]:
type(True)

In [ ]:
type(f)

**Be careful**, because `True` and `False` have special meanings in Python and _cannot_ be used as names. 

In [ ]:
# This doesn't work
3 = 4

In [ ]:
# This also doesn't work
True = 5

## `.where`

We've already seen how we can use `tbl.where()` to find rows that _exactly_ match what we're looking for. For example:

In [ ]:
sat.where('State', 'California')

But `tbl.where` is also capable of so much more! The second argument in `.where` can accept a **predicate**, which tells Python what condition to match rows on. Are few relevant predicates are:

| Predicate | Description |
| --- | --- |
| `are.equal_to(z)` | Is the value from the column equal to `z`? |
| `are.above(x)`, `are.below(x)` | Is the value from the column above/below `x`? |
| `are.between(x, y)` | Is the value from the column between `x` (inclusive) and `y` (exclusive)? |
| `are.containing(s)` | Does the value from the column contain the string `s`? |
| `are.contained_in(s)` | Is the value from the column inside the string/array s? |

You can also negate any of these predicates (i.e. find the opposite) by adding `not_` on the front of any of their function names (e.g. `are.not_equal_to(z)`).

A full list of predicates can be found on the [Python Reference](https://data6.org/su22/reference/#tablewhere-predicates).

Let's see the power of `.where` in action:

In [ ]:
sat.where('Combined', are.above(1800))

In [ ]:
sat.where('State', are.equal_to('California'))

Note that `are.equal_to(z)` is the same as just passing in `z` itself as the second argument.

In [ ]:
sat.where('State', are.containing('Dakota'))

In [ ]:
sat.where('Math', are.between(580, 600))

### Multiple Conditions

We can match rows to multiple conditions/predicates by chaining `.where` method calls together. For example, we can look for states where the participation rate is above 20% and the average combined SAT score is above 1500.

In [ ]:
sat.where('Participation Rate', are.above(20)).where('Combined', are.above(1500))

In [ ]:
sat.where('Participation Rate', are.below(10)).where('Combined', are.above(1600))

We can have multiple different values to match to if we put then in an array and then use `are.contained_in`.

In [ ]:
deep_south = np.array(['Alabama', 'Georgia', 'Louisiana', 'Mississippi', 'South Carolina'])

In [ ]:
sat.where('State', are.contained_in(deep_south))

In [ ]:
sat.where('State', are.contained_in(deep_south)) \
   .where('Participation Rate', are.below(10)) \
   .where('Combined', are.above(1600))

**Just for fun:** consider the scatter plot of all states' participation rates and combined SAT scores. Does this scatter plot imply that lower participation _causes_ higher SAT scores? Or what is going on here?

In [ ]:
px.scatter(data_frame = sat.to_df(), 
           x = 'Combined', 
           y = 'Participation Rate', 
           hover_data = {'State': True},
           title = 'Participation Rate vs. Combined SAT Score for States in 2014')

### Quick Check 2

For this Quick Check we will return to the `wnba` data from previous lectures.

In [ ]:
wnba = Table.read_table('data/wnba-2020.csv').select('Player', 'Tm', 'Pos', 'G', 'PTS')
wnba

Fill in the code below so that the result is the average **PTS** scored last season by **forwards** (players whose `Pos` is “F”) who **played 20 or more games**.

In [ ]:
wnba.where(____, ____).where('G',____).column(____).mean() # Replace the blanks with your answer